In [54]:
r_template_1 = """---
title: "TITLE"
output: html_document
date: "2025-01-27"
---

```{r}
library(ggplot2)
```

```{r}
questions = read.csv("data/INPUTFILE", sep="\t")
```
"""

r_template_2 = """
## Evaluate MODEL on DATASET

```{r}
results = read.csv("output/OUTPUTFILE", sep="\t")
```

```{r}
results$y = questions$y
```

```{r}
model = lm(results$y_pred ~ questions$y)
summary(model)
```

```{r}
ggplot(results, aes(x = y, y = y_pred) ) + geom_point() + geom_smooth(method = "lm") + ggtitle("DATASET: predictions of model\\nMODEL\\nvs ground truth (y)")
```"""

In [55]:
import os, re

In [56]:
output_files = set()
input_files= set()
for file in os.listdir("output"):
    if file.endswith("csv") and "continuous" in file:
        output_files.add(file)
for file in os.listdir("data"):
    if file.endswith("csv") and "continuous" in file:
        input_files.add(file)

In [57]:
for input_file in sorted(input_files):
    dataset_name = re.sub('_', ' ', re.sub(r'.csv$', '', input_file)).title()
    md = re.sub("TITLE", "Evaluation of models on dataset: " + dataset_name, r_template_1)
    md = re.sub("INPUTFILE", input_file, md)
    for output_file in sorted(output_files):
        if input_file in output_file:
            model_name = re.sub('_model_on_.+', '', output_file)
            
            new_segment = re.sub("OUTPUTFILE", output_file, r_template_2)
            new_segment = re.sub("MODEL", model_name, new_segment)
            new_segment = re.sub("MODEL", model_name, new_segment)
            new_segment = re.sub("DATASET", dataset_name, new_segment)
            new_segment = re.sub("DATASET", dataset_name, new_segment)
            
            md += new_segment
    
    with open("evaluate_" + re.sub('_continuous.csv', "", input_file) + ".Rmd", "w", encoding="utf-8") as f:
        f.write(md)